# Running Constraint Scan

<span style="color:red">
    <font size="6">
Remember to run for both BLUE and WHITE light
    </font>
</span>

In [1]:
# | default_exp runconstraintscan

This notebook can either be run as a notebook, or using nbdev a python script is produced that can be run in the command line for easier running on e.g. a cluster. It is a notebook in the nbs directory and exported as a .py file in the mmon-gcm library.

In [2]:
# | export

import sys
import os

import cobra
import mmon_gcm.constraintscan  # from x import y syntax doesn't work because of nbdev export format
import pandas as pd
from pandarallel import pandarallel

In [3]:
# | export

test = False

In [4]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is also where to change whether the light is blue or white if you want to run in the notebook

sys.argv = [
    "script_name",
    "../outputs/constraint_scan/constraint_scan_results_blue.csv",
    "../models/4_stage_GC.json",
    "../outputs/constraint_scan/constraints_df.csv",
    "blue",
    "6",
]

In [5]:
# | export

results_path = sys.argv[1]
model_path = sys.argv[2]
constraints_csv = sys.argv[3]
light_colour = sys.argv[4]
no_cores = int(sys.argv[5])

In [6]:
# | export
constraints = pd.read_csv(constraints_csv, index_col=[0], header=[0])
model = cobra.io.load_json_model(model_path)

**If you are running from notebook you'll want to comment out the next cell! It's here so that running nbdev tests doesn't take too long, as they test running the entire notebook. It won't be exported so running as a script will be fine**

In [7]:
#test == True
#constraints = constraints.iloc[:10]  # so tests don't take too long

In [8]:
constraints.head()

,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,R_ch_vol,L_air,L_epidermis,Vac_frac,...,R,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase
0,0.815093,0.000194,1.0,2.262641e-12,0.809297,0.179970,0.191527,0.240879,0.214860,0.858645,...,0.08205,4.484392e+08,1.933548,0.992641,7.886553e-14,1.678654e-13,0.033679,2.321339,11.318979,10.816668
1,0.923180,0.000190,1.0,1.250880e-12,0.889162,0.093534,0.195869,0.337197,0.196606,0.837767,...,0.08205,1.064044e+09,2.025073,0.946228,5.400144e-14,1.664063e-13,0.023524,2.500598,9.754649,0.549826
2,0.830507,0.000220,1.0,5.035745e-13,0.821060,0.167889,0.204824,0.331556,0.205674,0.816618,...,0.08205,5.758277e+08,2.141889,0.972835,6.579620e-14,2.457118e-13,0.034062,2.802180,3.338120,7.823891
3,0.880998,0.000192,1.0,8.629192e-13,0.866582,0.051244,0.204472,0.309538,0.169957,0.813726,...,0.08205,3.851195e+08,2.077401,0.940848,5.747590e-14,1.515328e-13,0.029770,3.399462,9.936390,12.606738
4,0.915597,0.000220,1.0,7.391447e-13,0.846358,0.059969,0.193449,0.352066,0.238671,0.810491,...,0.08205,1.046353e+09,2.396012,0.817798,7.654181e-14,1.652973e-13,0.028420,3.305233,7.650706,10.970481


In [9]:
print(f"Number of constraints = {len(constraints.index)}")

Number of constraints = 968


### Breakdown into batches of constraints

In [10]:
# | export

# Define the batch size
batch_size = 100

df = constraints
# Calculate the number of batches
num_batches = len(df) // batch_size + 1

# Iterate through batches
for batch_number in range(num_batches):
    # Calculate the start and end indices for the current batch
    start_index = batch_number * batch_size
    end_index = (batch_number + 1) * batch_size

    # Slice the DataFrame for the current batch
    current_batch = df.iloc[start_index:end_index]

    if not os.path.exists(f"{results_path[:-4]}_{batch_number+1}.csv"):
        pandarallel.initialize(nb_workers=no_cores, progress_bar=False)
        #scan_solution = constraints.parallel_apply(
        scan_solution = current_batch.parallel_apply(
            mmon_gcm.constraintscan.solve_model_for_constraint_scan,
            args=([model, light_colour, False]),
            axis=1,
        )

        solution_df = pd.DataFrame()
        for i, solution in scan_solution.items():
            # this is to catch infeasible solutions
            try:
                solution_df = solution_df.append(
                    solution.to_frame().loc[:, "fluxes"], ignore_index=True
                )
            except:
                solution_df = solution_df.append(pd.Series(dtype=float), ignore_index=True)

        solution_df.to_csv(f"{results_path[:-4]}_{batch_number+1}.csv")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 0
Solving model 17
Solving model 34
Solving model 52
Solving model 70
Solving model 89
Solved model 0
Solved model 17
Solved model 34
Solved model 52
Solved model 70
Solved model 89
Solving model 1
Solving model 18
Solving model 35
Solving model 53
Solving model 71
Solving model 90
Solved model 1
Solved model 18
Solved model 53
Solved model 35
Solved model 71
Solving model 2Solved model 90

Solving model 19
Solving model 55
Solving model 36
Solving model 73
Solving model 91
Solved model 2
Solved model 19
Solving model 3
Solved model 55
Solved model 36
Solved model 73
Solved model 91
Solving model 20
Solving model 56
Solving model 37
Solving model 74
Solving model 92
Solved model 3
Solved model 20
Solving model 4
Solved model 74
Solved model 37
Solving model 21
Solved model 56
Solved model 92
Solving model 76
Solving model 38
Solving

/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 122
Solving model 105
Solving model 139
Solving model 156
Solving model 174
Solving model 191
Solved model 122
Solving model 123
Solved model 139Solved model 105

Solved model 191
Solved model 156
Solving model 106
Solving model 140
Solving model 192
Solved model 174
Solving model 157
Solving model 175
Solved model 123
Solved model 192
Solved model 106
Solved model 140
Solving model 124
Solved model 157
Solving model 193
Solved model 175
Solving model 141Solving model 107

Solving model 158
Solving model 176
Solved model 124
Solved model 107
Solved model 193
Solved model 141
Solving model 125
Solved model 158
Solved model 176
Solving model 108
Solving model 195
Solving model 142
Solving model 159
Solving model 177
Solved model 125
Solved model 108
Solved model 195
Solving model 126
Solving model 109
Solved model 142
Solving model 19

...

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 846Solving model 828

Solving model 863
Solving model 880
Solving model 898
Solving model 914
Solved model 846
Solved model 828
Solved model 898
Solving model 847
Solving model 829
Solving model 899
Solved model 880
Solved model 863
Solved model 914
Solving model 915
Solving model 881Solving model 864

Solved model 847
Solved model 829
Solved model 915
Solved model 881
Solved model 899
Solving model 848
Solving model 830
Solved model 864
Solving model 917
Solving model 882
Solving model 900
Solving model 865
Solved model 917
Solved model 830
Solved model 848
Solved model 882
Solved model 900
Solved model 865
Solving model 918
Solving model 831
Solving model 849
Solving model 883
Solving model 901
Solving model 866
Solved model 918
Solved model 831
Solved model 883
Solving model 919
Solved model 849
Solving model 832
Solved model 866

/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model 931
Solving model 944
Solving model 956
Solving model 967
Solving model 978
Solving model 989
Solved model 931
Solved model 944
Solved model 989
Solving model 932
Solving model 945
Solved model 956
Solved model 967
Solved model 978
Solving model 990
Solving model 957
Solving model 968
Solving model 979
Solved model 932
Solved model 990
Solved model 945
Solved model 957
Solving model 933
Solved model 968
Solved model 979
Solving model 991
Solving model 946
Solving model 958
Solving model 969
Solving model 980
Solved model 991
Solved model 933
Solved model 958
Solving model 992
Solved model 946
Solved model 980
Solved model 969
Solving model 934
Solving model 959
Solving model 947
Solving model 981
Solving model 970
Solved model 992
Solved model 934
Solved model 959
Solving model 993
Solved model 970
Solving model 935
Solved model 947

/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  solution_df = solution_df.append(
/tmp/ipykernel_19070/2574100191.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [11]:
if os.path.exists(f"{results_path[:-4]}_{num_batches}.csv"):
    directory_path = os.path.dirname(results_path)
    # Get a list of all files in the directory
    all_files = os.listdir(directory_path)
    # Filter the list to include
    csv_files = [file for file in all_files if file.startswith(f"{results_path[:-4]}_".rsplit("/")[-1])]

    # Sort the list using a lambda function and underscore as separator
    csv_files = sorted(csv_files, key=lambda x: int(x.rsplit('_')[-1].split('.')[0]))

    # Initialize an empty list to store DataFrames
    dfs = []

    # Read each CSV file into a DataFrame and append it to the list
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path)
        dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    # Remove 'Unnamed: 0' column
    if 'Unnamed: 0' in combined_df.columns:
        combined_df = combined_df.drop('Unnamed: 0', axis=1)
        
    # Print or use the combined DataFrame as needed
    combined_df.to_csv(results_path)
    
    for csv_file in list(csv_files):
        file_path = os.path.join(directory_path, csv_file)
        if os.path.exists(file_path):
            os.remove(file_path)

In [12]:
combined_df.shape

(968, 7101)